In [425]:
%run SharedParams.ipynb
%run MerkleTree.ipynb
import ast

proof = proof_example
commitments = []
fiat_shamir = FiatShamir()

def parse_and_commit_hash(arg_name):
    arg = proof.pop(0)
    assert arg.startswith(arg_name + ":")
    arg = bytes.fromhex(arg[len(arg_name) + 1:])
    commitments.append(arg)
    return arg


eval_merkle_root = parse_and_commit_hash("eval_merkle")

beta0, beta1, beta2, beta3, beta4, beta5 = (fiat_shamir.get_random_int(commitments) % P for i in range(6))

cp_eval_merkle_root = parse_and_commit_hash("cp_eval_merkle")







In [426]:
def parse_felt_and_auth(arg_name, index, merkle_root, tree_height):
    data = proof.pop(0)
    path = proof.pop(0)
    
    assert data.startswith(arg_name + ":")
    data = data[len(arg_name) + 1:]
    assert path.startswith("path:")
    path = ast.literal_eval(path[5:])
    MerkleTree.auth_path(data, index, path, merkle_root, tree_height)
    
    return int(data) % P


In [427]:
layers_roots = []
layers_betas = []
for layer in range(int(math.log(gamma_order, 2)) - 1):
    layers_betas.append(fiat_shamir.get_random_int(commitments))
    layers_roots.append(parse_and_commit_hash("layer_root"))

layers_betas.append(fiat_shamir.get_random_int(commitments))

last_layer_const = proof.pop(0)
assert last_layer_const.startswith("last_layer_const:")
last_layer_const = int(last_layer_const[17: ])
commitments.append(last_layer_const)


In [428]:
for i in range(N_FRI_QUERIES):
    print(f"running Fri query {i}")
    tree_height = int(math.log(g_order, 2))    
    first_index = fiat_shamir.get_random_int(commitments) % g_order
    second_index = (first_index + int(g_order / 2)) % g_order
    for j in range(3):
        parse_felt_and_auth("mask_eval", (first_index + j * RS_factor) % g_order, eval_merkle_root, tree_height)
        
    cp_first_eval = parse_felt_and_auth("layer_first_eval", first_index, cp_eval_merkle_root, tree_height)
    cp_second_eval = parse_felt_and_auth("layer_second_eval", second_index, cp_eval_merkle_root, tree_height)
    
    # TODO - check the mask against the cp values. (build polynom)
    
    # Get and authenticate the plus and minus evaluations.
    layers_first_eval = []
    layers_second_eval = []
    layer_size = g_order
    for layer_root in layers_roots:
        tree_height -= 1
        layer_size = int(layer_size / 2)
        layer_first_index = first_index % layer_size
        layer_second_index = int(first_index - int(layer_size / 2)) % layer_size
        layers_first_eval.append(parse_felt_and_auth("layer_first_eval", layer_first_index, layer_root, tree_height))
        layers_second_eval.append(parse_felt_and_auth("layer_second_eval", layer_second_index, layer_root, tree_height))

    # TODO - add layer evaluation as below to CP <-> CP1
    
    # Check Fri layers evaluations.
    dilution = 2
    for layer_index in range(len(layers_roots)):
        layer_size = int(g_order / dilution)
        cur_index = (dilution * first_index) % int(g_order / 2)
        layer_first_index = first_index % layer_size
        layer_second_index = int(first_index - int(layer_size / 2)) % layer_size
        
        plus_eval = layers_first_eval[layer_index] if layer_first_index < layer_second_index else layers_second_eval[layer_index]
        minus_eval = layers_first_eval[layer_index] if layer_first_index > layer_second_index else layers_second_eval[layer_index]
        
        even_part = (F_gen * plus_eval + minus_eval) / 2
        odd_part = (F_gen * plus_eval - minus_eval) / (2 * w**dilution * g**(cur_index))
        
        if layer_index == len(layers_roots) - 1:
            expected_value = F_gen * last_layer_const
        else:
            expected_value = F_gen * layers_first_eval[layer_index + 1]
        assert expected_value == even_part + layers_betas[layer_index + 1] * odd_part
            
        dilution *= 2

        
assert len(proof) == 0
print("Amen :)")

running Fri query 0
running Fri query 1
Amen :)


In [429]:
print(commitments)

[b'W\x07a\xf5\x88\x81y`\xdcmqqw\x88\x1d\x9b\xe5\n\xe3\xb5\x8d\x8b\xf7\xd3\xb3\xd2\xa0\xbe\x85q\xfb\xce', b'\xc0\xcbg\x06\x83O\x1e~n@\xff\x92\xfdI\xb6\x88\x8a[0\x01\x04Ql\xf5#\x8d\xc1_\xf3|\x8b\x99', b'BL>\x85K\xc2X\xa7\x15*}\t\xff\xc0+\x85\x92nq\x059IX\xb6o\xe54\xa9\xe0KM\x19', b'\xd5\x82\xffAU\x1f63\xd4\xf0\xb1\x16\x9d\x99\x7f\xbc3\xbe\x8f\xdb`\xa7n\x0e\xb6u\xdc\xb4\xec\x19\xbd ', b'\xb1@\x03\x05!\xcb\xbc\xfd\x9d?\x1a  ,\x86(\xb5fS\x81V\x9f\x08\x18\xf0\xdf\x8ch\x17\xf7\x986', b'\xd7\xa1\x9a\xa6\x99\x05\xad\x7f\x92\x1d\xea\xdd1\xf0\x12.\x80\x9c\xad|\xea\x9b\xc3\x1f#\n\xba\x95\x9b&\x14\xb4', b'\x1f\x03\x82,>\xb8\x12\x84\x15\x90L\xc7\xe1\x08B\x91\x19\x9f%MJ\x93dT\x9c\xdd\x19zx\xa0\x05p', b'\xb8\x83\xa4\xa6\x9b\x05\x1d\x80\xae#\xfd\x97\x9545i\x82t$\xf8G\xa2\x16u\xd3\xde\x11\xa6\x90\x154\xac', b'k\xd4\x85a\xe0O\x88\xc8uX\x07\xe7\r\xe7y\x87q\x1f\xd4\xc0\xa6B\x00\xfb\x12Pk\xaa\x8a\xc1ma', b'\t\xc1-\xc9\x93{\xa5\xcb\xf4w\x0c\xd3\xca\xd5,-\x14.\xd3^\xfdN\x94w\x8bW\xfd\ny\xaa\x1c\xe3', b'\x90F\